In [1]:
import spacy

spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]



In [2]:
tokenized = spacy_en.tokenizer('I am a graduate student')

for i, token in enumerate(tokenized):
    print(i, token.text)

0 I
1 am
2 a
3 graduate
4 student


In [3]:
# 최신 torchtext API 사용
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader, Dataset
import torch

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 중인 device: {device}")

# 토크나이저 함수 (기존과 동일)
def tokenize_de_new(text):
    return ['<sos>'] + tokenize_de(text.lower()) + ['<eos>']

def tokenize_en_new(text):
    return ['<sos>'] + tokenize_en(text.lower()) + ['<eos>']


사용 중인 device: cuda


In [4]:
# import ssl
# ssl._create_default_https_context = ssl._create_unverified_context

In [5]:
# 직접 파일을 읽어서 데이터셋 생성
import os

def read_data_files(base_path, split_name):
    de_file = os.path.join(base_path, f"{split_name}.de")
    en_file = os.path.join(base_path, f"{split_name}.en")
    
    with open(de_file, 'r', encoding='utf-8') as f_de, \
         open(en_file, 'r', encoding='utf-8') as f_en:
        de_lines = [line.strip() for line in f_de]
        en_lines = [line.strip() for line in f_en]
    
    return list(zip(de_lines, en_lines))

# 데이터 로딩
data_path = '../.data/multi30k'
train_data = read_data_files(data_path, 'train')
valid_data = read_data_files(data_path, 'val')

# test 파일명 처리
try:
    test_data = read_data_files(data_path, 'test_2016_flickr')
except:
    test_data = read_data_files(data_path, 'test2016')

print(f"학습 데이터: {len(train_data)}개")
print(f"검증 데이터: {len(valid_data)}개") 
print(f"테스트 데이터: {len(test_data)}개")

학습 데이터: 29000개
검증 데이터: 1014개
테스트 데이터: 1000개


In [6]:
# 데이터 샘플 확인
print("30번째 예시:")
print("Source (독일어):", train_data[30][0])
print("Target (영어):", train_data[30][1])

# 토큰화된 결과 확인
print("\n토큰화된 결과:")
print("Source (독일어):", tokenize_de_new(train_data[30][0]))
print("Target (영어):", tokenize_en_new(train_data[30][1]))

30번째 예시:
Source (독일어): Ein Mann, der mit einer Tasse Kaffee an einem Urinal steht.
Target (영어): A man standing at a urinal with a coffee cup.

토큰화된 결과:
Source (독일어): ['<sos>', 'ein', 'mann', ',', 'der', 'mit', 'einer', 'tasse', 'kaffee', 'an', 'einem', 'urinal', 'steht', '.', '<eos>']
Target (영어): ['<sos>', 'a', 'man', 'standing', 'at', 'a', 'urinal', 'with', 'a', 'coffee', 'cup', '.', '<eos>']


In [7]:
# 수정된 어휘집 구축 (구버전 torchtext 호환)
from collections import Counter

def yield_tokens_de(data_iter):
    for de_text, en_text in data_iter:
        yield tokenize_de_new(de_text)

def yield_tokens_en(data_iter):
    for de_text, en_text in data_iter:
        yield tokenize_en_new(en_text)

# 특수 토큰 정의
UNK_TOKEN = '<unk>'
PAD_TOKEN = '<pad>'  
SOS_TOKEN = '<sos>'
EOS_TOKEN = '<eos>'

# 수동으로 어휘집 구축 (구버전 호환)
def build_vocab_manual(token_iterator, min_freq=2):
    # 토큰 빈도 계산
    counter = Counter()
    for tokens in token_iterator:
        counter.update(tokens)
    
    # 특수 토큰을 먼저 추가
    vocab_tokens = [UNK_TOKEN, PAD_TOKEN, SOS_TOKEN, EOS_TOKEN]
    
    # min_freq 이상인 토큰 추가 (특수 토큰 제외)
    for token, freq in counter.items():
        if freq >= min_freq and token not in vocab_tokens:
            vocab_tokens.append(token)
    
    # 토큰-인덱스 매핑 생성
    stoi = {token: idx for idx, token in enumerate(vocab_tokens)}
    itos = vocab_tokens
    
    return vocab_tokens, stoi, itos

# 독일어 어휘집 구축
de_tokens, de_stoi, de_itos = build_vocab_manual(yield_tokens_de(train_data), min_freq=2)
print(f"독일어 어휘집 크기: {len(de_tokens)}")

# 영어 어휘집 구축
en_tokens, en_stoi, en_itos = build_vocab_manual(yield_tokens_en(train_data), min_freq=2)
print(f"영어 어휘집 크기: {len(en_tokens)}")

# 간단한 어휘집 클래스 (torchtext 없이)
class SimpleVocab:
    def __init__(self, tokens, stoi, itos):
        self.tokens = tokens
        self.stoi = stoi
        self.itos = itos
        self.unk_token = UNK_TOKEN
        self.pad_token = PAD_TOKEN
    
    def __len__(self):
        return len(self.tokens)
    
    def __getitem__(self, token):
        return self.stoi.get(token, self.stoi[self.unk_token])
    
    def __call__(self, tokens):
        """토큰 리스트를 인덱스 리스트로 변환"""
        if isinstance(tokens, str):
            return self.stoi.get(tokens, self.stoi[self.unk_token])
        return [self.stoi.get(token, self.stoi[self.unk_token]) for token in tokens]

# BucketIterator 호환 래퍼 클래스
class VocabWrapper:
    def __init__(self, simple_vocab):
        self.vocab = simple_vocab
        self.pad_token = simple_vocab.pad_token
        self.stoi = simple_vocab.stoi
        self.itos = simple_vocab.itos
    
    def __len__(self):
        return len(self.vocab)
    
    def __getitem__(self, token):
        return self.vocab[token]

# 어휘집 생성
de_simple_vocab = SimpleVocab(de_tokens, de_stoi, de_itos)
en_simple_vocab = SimpleVocab(en_tokens, en_stoi, en_itos)

SRC = VocabWrapper(de_simple_vocab)
TRG = VocabWrapper(en_simple_vocab)

print(f"PAD 토큰 인덱스 - SRC: {SRC.stoi[PAD_TOKEN]}, TRG: {TRG.stoi[PAD_TOKEN]}")
print(f"UNK 토큰 인덱스 - SRC: {SRC.stoi[UNK_TOKEN]}, TRG: {TRG.stoi[UNK_TOKEN]}")
print(f"SOS 토큰 인덱스 - SRC: {SRC.stoi[SOS_TOKEN]}, TRG: {TRG.stoi[SOS_TOKEN]}")
print(f"EOS 토큰 인덱스 - SRC: {SRC.stoi[EOS_TOKEN]}, TRG: {TRG.stoi[EOS_TOKEN]}")


독일어 어휘집 크기: 7853
영어 어휘집 크기: 5893
PAD 토큰 인덱스 - SRC: 1, TRG: 1
UNK 토큰 인덱스 - SRC: 0, TRG: 0
SOS 토큰 인덱스 - SRC: 2, TRG: 2
EOS 토큰 인덱스 - SRC: 3, TRG: 3


In [8]:
from torch.nn.utils.rnn import pad_sequence

# 수정된 Dataset과 DataLoader (BucketIterator 대체)
class TranslationDataset(Dataset):
    def __init__(self, data, src_vocab, trg_vocab, tokenize_src, tokenize_trg):
        self.data = data
        self.src_vocab = src_vocab
        self.trg_vocab = trg_vocab
        self.tokenize_src = tokenize_src
        self.tokenize_trg = tokenize_trg
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        src_text, trg_text = self.data[idx]
        
        # 토큰화
        src_tokens = self.tokenize_src(src_text)
        trg_tokens = self.tokenize_trg(trg_text)
        
        # 인덱스로 변환 (수정된 방식)
        src_indices = self.src_vocab.vocab(src_tokens)
        trg_indices = self.trg_vocab.vocab(trg_tokens)
        
        return torch.tensor(src_indices), torch.tensor(trg_indices)

# collate 함수 (배치 처리)
def collate_fn(batch):
    src_batch, trg_batch = zip(*batch)
    
    # 패딩 처리
    # pad_sequence는 서로 길이가 다른 시퀀스(문장)들을 동일한 길이로 맞춰주는 함수입니다.
    # 각 시퀀스의 길이가 다를 때, 가장 긴 시퀀스에 맞춰 나머지 시퀀스에 PAD 토큰을 추가하여 패딩합니다.
    # batch_first=True는 배치 차원이 첫 번째가 되도록 만듭니다.
    # padding_value는 패딩에 사용할 값(PAD 토큰의 인덱스)을 지정합니다.

    src_batch = pad_sequence(src_batch, batch_first=True, padding_value=SRC.stoi[PAD_TOKEN])
    trg_batch = pad_sequence(trg_batch, batch_first=True, padding_value=TRG.stoi[PAD_TOKEN])
    
    return src_batch, trg_batch

# Dataset 생성
BATCH_SIZE = 128

train_dataset = TranslationDataset(train_data, SRC, TRG, tokenize_de_new, tokenize_en_new)
valid_dataset = TranslationDataset(valid_data, SRC, TRG, tokenize_de_new, tokenize_en_new)
test_dataset = TranslationDataset(test_data, SRC, TRG, tokenize_de_new, tokenize_en_new)

# DataLoader 생성
train_iterator = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    collate_fn=collate_fn,
    pin_memory=True if torch.cuda.is_available() else False
)

valid_iterator = DataLoader(
    valid_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=collate_fn,
    pin_memory=True if torch.cuda.is_available() else False
)

test_iterator = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    collate_fn=collate_fn,
    pin_memory=True if torch.cuda.is_available() else False
)

print(f"DataLoader 생성 완료!")
print(f"학습 배치 수: {len(train_iterator)}")
print(f"검증 배치 수: {len(valid_iterator)}")
print(f"테스트 배치 수: {len(test_iterator)}")


DataLoader 생성 완료!
학습 배치 수: 227
검증 배치 수: 8
테스트 배치 수: 8


In [9]:
import torch

print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"CUDA 버전: {torch.version.cuda}")
print(f"cuDNN 버전: {torch.backends.cudnn.version()}")

if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")
else:
    print("CUDA를 사용할 수 없습니다.")

PyTorch 버전: 2.7.1+cu118
CUDA 사용 가능: True
CUDA 버전: 11.8
cuDNN 버전: 90100
GPU 개수: 1
GPU 이름: NVIDIA GeForce RTX 4070 Ti


In [10]:
# GPU 에러 디버깅을 위해 CPU에서 먼저 테스트
print("=== GPU 에러 디버깅 ===")

# 1. 어휘집 인덱스 범위 체크
print(f"Source vocab 크기: {len(SRC)}")
print(f"Target vocab 크기: {len(TRG)}")
# 2. 샘플 배치에서 인덱스 범위 확인
for i, batch in enumerate(train_iterator):
    src, trg = batch
    print(f"\n배치 {i+1}:")
    print(f"Source 최대 인덱스: {src.max().item()}, 최소 인덱스: {src.min().item()}")
    print(f"Target 최대 인덱스: {trg.max().item()}, 최소 인덱스: {trg.min().item()}")
    
    # 범위 체크
    if src.max().item() >= len(SRC):
        print(f"❌ Source 인덱스 오류: {src.max().item()} >= {len(SRC)}")
    if trg.max().item() >= len(TRG):
        print(f"❌ Target 인덱스 오류: {trg.max().item()} >= {len(TRG)}")
    
    if i >= 2:  # 처음 3개 배치만 체크
        break

print("\n=== Teacher forcing 처리 체크 ===")
src, trg = next(iter(train_iterator))
print(f"Original trg shape: {trg.shape}")
print(f"trg[:, :-1] shape: {trg[:, :-1].shape}")  # 마지막 토큰 제거
print(f"trg[:, 1:] shape: {trg[:, 1:].shape}")   # 첫 번째 토큰 제거


=== GPU 에러 디버깅 ===
Source vocab 크기: 7853
Target vocab 크기: 5893

배치 1:
Source 최대 인덱스: 7808, 최소 인덱스: 0
Target 최대 인덱스: 5882, 최소 인덱스: 0

배치 2:
Source 최대 인덱스: 7795, 최소 인덱스: 0
Target 최대 인덱스: 5885, 최소 인덱스: 0

배치 3:
Source 최대 인덱스: 7834, 최소 인덱스: 0
Target 최대 인덱스: 5759, 최소 인덱스: 0

=== Teacher forcing 처리 체크 ===
Original trg shape: torch.Size([128, 34])
trg[:, :-1] shape: torch.Size([128, 33])
trg[:, 1:] shape: torch.Size([128, 33])


## 1. Multi-head attention 아키텍쳐

- 세 가지 요소를 입력을 받는 attention
    - 쿼리 (Queries)
    - 키 (Keys)
    - 값 (Values)
- 하이퍼 파라미터
    - hidden_dim : 하나의 단어에 대한 임베딩 차원
    - n_heads : 헤드의 개수 = scaled dot-product attention의 개수
    - dropout_ratio : 드롭아웃 비율

In [11]:
import torch.nn as nn

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, dropout_ratio, device):
        super().__init__()

        assert hidden_dim % n_heads == 0

        self.hidden_dim = hidden_dim # 임베딩 차원
        self.n_heads = n_heads # 헤드(head)의 개수, 서로 다른 attention 컨셉 수
        self.head_dim = hidden_dim // n_heads # 각 헤드의 차원

        self.fc_q = nn.Linear(hidden_dim, hidden_dim) # Query 값에 적용될 FC 레이어
        self.fc_k = nn.Linear(hidden_dim, hidden_dim) # Key 값에 적용될 FC 레이어
        self.fc_v = nn.Linear(hidden_dim, hidden_dim) # Value 값에 적용될 FC 레이어

        self.fc_o = nn.Linear(hidden_dim, hidden_dim) # 최종 결과에 적용될 FC 레이어

        self.dropout = nn.Dropout(dropout_ratio)
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)  

    def forward(self, query, key, value, mask = None):

        batch_size = query.shape[0]

        # query : [batch_size, query_len, hidden_dim]
        # key : [batch_size, key_len, hidden_dim]
        # value : [batch_size, value_len, hidden_dim]

        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)

        # hidden_dim => n_heads x head_dim 형태로 변형 
        # view: 텐서의 shape을 바꿔줌. -1은 자동 계산.
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0,2,1,3)

        # Q : [batch_size, n_heads, query_len, head_dim]
        # K : [batch_size, n_heads, key_len, head_dim]
        # V : [batch_size, n_heads, value_len, head_dim]

        # Attention Energy 계산
        energy = torch.matmul(Q, K.permute(0,1,3,2)) / self.scale

        # 마스크 (mask)를 사용하는 경우
        if mask is not None:
            energy = energy.masked_fill(mask ==0, -1e10)

        # attention 스코어에 대한 softmax 연산
        # attention : [batch)size, n_heads, query_len, key_len]
        attention = torch.softmax(energy, dim = -1)

        # Scaled Dot-Product Attention을 계산
        # x : [batch_size, n_heads, query_len, head_dim]
        x = torch.matmul(self.dropout(attention), V)

        # permute는 텐서 차원 순서 변경, contiguous는 메모리 연속성 보장 함수
        # x : [batch_size, query_len, n_heads, head_dim]
        x = x.permute(0,2,1,3).contiguous()
        
        # x : [batch_size, query_len, hidden_dim]
        x = x.view(batch_size, -1, self.hidden_dim)

        x = self.fc_o(x)

        return x, attention
        

## 2. Position-wise Feedforward 아키텍쳐
- 입력과 출력이 동일
- 하이퍼 파라미터
    - hidden_dim : 하나의 단어에 대한 임베딩 차원
    - pf_dim : feedforward 레이어에서의 내부 임베딩 차원
    - dropout_ratio : 드롭아웃 비율

In [12]:
class PositionwiseFeedforwardLayer(nn.Module):

    def __init__(self, hidden_dim, pf_dim, dropout_ratio):
        super().__init__()

        self.fc_1 = nn.Linear(hidden_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hidden_dim)

        self.dropout = nn.Dropout(dropout_ratio)

    def forward(self, x):
        x = self.dropout(torch.relu(self.fc_1(x)))
        x = self.fc_2(x)
        
        return x

## 3-1. Encoder 레이어 아키텍쳐
- 하나의 인코더 레이어에 대해 정의
    - 입력과 출력의 차원이 같다.
    - 이 특징을 사용해 트랜스포머의 인코더는 인코더 레이어를 여러 번 중첩해서 사용함.

- 하이퍼 파라미터
    - hidden_dim : 하나의 단어에 대한 임베딩 차원
    - n_heads : 헤드의 개수 = scaled dot-product attention의 개수
    - dropout_ratio : 드롭아웃 비율

- <pad> 토큰에 대해 마스크 값을 0으로 설정함

In [13]:
class EncoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        self.self_attn = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 하나의 임베딩이 복제되어 Q, K, V 로 입력되는 방식

    def forward(self, src, src_mask):

        # src : [batch_size, src_len, hidden_dim]
        # src_mask : [batch_size, src_len]

        # self attention
        # 필요 시 마스크 행렬을 이용해 어텐션할 단어 조절 가능

        # self.self_attn은 MultiHeadAttentionLayer를 의미하며, 이 레이어는 입력값으로 Q(Query), K(Key), V(Value), 그리고 마스크(src_mask)를 받습니다.
        # 트랜스포머 인코더에서는 self-attention이므로 Q, K, V 모두 동일한 입력(src)을 사용합니다.

        # src_mask는 패딩 토큰 등에 대한 마스킹을 위해 사용됩니다.
        # self.self_attn의 반환값은 (attention_output, attention_weights)로, 여기서는 attention_output만 사용하고 attention_weights는 사용하지 않으므로 _로 처리합니다.
        
        # _src는 self-attention 레이어를 통과한 결과(즉, 어텐션을 적용한 후의 임베딩)이고,
        # src는 아직 self-attention을 통과하지 않은 입력 임베딩입니다.
        
        # self-attention의 출력(_src)에 드롭아웃을 적용한 뒤, 입력(src)와 더해주고,
        # 그 결과를 LayerNorm에 통과시켜서 최종적으로 src를 업데이트합니다.

        _src, _ = self.self_attn(src, src, src, src_mask)
        src = self.self_attn_layer_norm(src + self.dropout(_src))

        _src = self.positionwise_feedforward(src)
        src = self.ff_layer_norm(src + self.dropout(_src))

        return src
        

## 3-2. Encoder 아키텍쳐
- 전체 인코터 아키텍쳐의 정의
- 하이퍼 파라미터
    - input_dim : 하나의 단어에 대한 원-핫 인코딩 차원
    - hidden_dim : 하나의 단어에 대한 임베딩 차원
    - n_layers : 내부적으로 사용할 인코더 레이어의 개수
    - n_heads: 헤드의 개수 = scaled dot-product attention의 개수
    - pf_dim : feedforward 레이어에서의 내부 임베딩 차원
    - dropout_ratio : 드롭아웃 비율
    - max_length : 문장 내 최대 단어 개수
- 위치 임베딩 (positional embedding)을 학습하는 형태로 구현
- <pad> 토큰에 대해 마스크 값을 0으로 설정

In [14]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length = 100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(input_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([EncoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])

        self.dropout = nn.Dropout(dropout_ratio)
        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, src, src_mask):
        batch_size = src.shape[0]
        src_len = src.shape[1]

        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))

        for layer in self.layers:
            src = layer(src, src_mask)

        return src

## 4-1. Decoder 레이어 아키텍쳐
- 하나의 디코더 레이어에 대해 정의
    - 입력과 출력의 차원이 같음.
    - 이 특징을 이용해 인코더와 마찬가지로 디코더 레이어를 여러 번 중첩해서 사용.
    - 디코더 레이어에서는 두 개의 multi-head attention 레이어를 사용.
- 하이퍼 파라미터
    - hidden_dim : 하나의 단어에 대한 임베딩 차원
    - n_heads : 헤드의 개수 = scaled dot-product attention의 개수
    - pf_dim : feedforward 레이어에서의 내부 임베딩 차원
    - dropout_ratio : 드롭아웃 비율
- 소스 문장의 <pad> 토큰에 대해 마스크 값을 0으로 설정
- 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없게 (바로 이전 단어만 보도록) 마스크를 사용함.

In [15]:
class DecoderLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads, pf_dim, dropout_ratio, device):
        super().__init__()

        self.self_attn_layer_norm = nn.LayerNorm(hidden_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hidden_dim)

        self.ff_layer_norm = nn.LayerNorm(hidden_dim)
        
        self.self_attn = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)
        self.encoder_attn = MultiHeadAttentionLayer(hidden_dim, n_heads, dropout_ratio, device)

        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hidden_dim, pf_dim, dropout_ratio)
        self.dropout = nn.Dropout(dropout_ratio)

    # 인코더의 출력값(enc_src)을 attention 과정에서 사용하기 위해 입력값으로 받음
    def forward(self, trg, enc_src, trg_mask, src_mask):

        # 1. self attention -> 자기 자신에 대해 어텐션 적용
        _trg, _ = self.self_attn(trg, trg, trg, trg_mask)

        # dropout, residual connection & layer normalization
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))

        # 2. encoder attention -> 디코더의 쿼리를 이용해 인코더를 어텐션
        _trg, attention = self.encoder_attn(trg, enc_src, enc_src, src_mask)

        # dropout, residual connection & layer normalization
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))

        # 3. positionwise feedforward
        _trg = self.positionwise_feedforward(trg)

        # dropout, residual connection & layer normalization
        trg = self.ff_layer_norm(trg + self.dropout(_trg))

        return trg, attention



## 4-2. Decoder 아키텍쳐
- 전체 디코더 아키텍쳐를 정의
- 하이퍼 파라미터
    - output_dim : 하나의 단어에 대한 원-핫 인코딩 차원
    - hidden_dim : 하나의 단어에 대한 임베딩 차원
    - n_layers : 내부적으로 사용할 인코더 레이어의 개수
    - n_heads : 헤드의 개수 = scaled dot-product attention의 개수
    - pf_dim : feedforward 레이어에서의 내부 임베딩 차원
    - dropout_ratio : 드롭아웃 비율
    - max_length : 문장 내 최대 단어 개수
- 위치 임베딩 (positional embedding)을 학습하는 형태로 구현
- Seq2Seq 과 마찬가지로 실제 추론 (inference) 과정에서는 디코더를 반복적으로 넣을 필요가 있음.
    - 학습 시기에는 한 번에 출력 문장을 구해 학습함. 
- 소스 문장의 <pad> 토큰에 대해 마스크 값을 0으로 설정.
- 타겟 문장에서 각 단어는 다음 단어가 무엇인지 알 수 없도록 만들기 위해 마스크를 사용.

In [16]:
class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device, max_length = 100):
        super().__init__()

        self.device = device

        self.tok_embedding = nn.Embedding(output_dim, hidden_dim)
        self.pos_embedding = nn.Embedding(max_length, hidden_dim)

        self.layers = nn.ModuleList([DecoderLayer(hidden_dim, n_heads, pf_dim, dropout_ratio, device) for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hidden_dim, output_dim)

        self.dropout = nn.Dropout(dropout_ratio)

        self.scale = torch.sqrt(torch.FloatTensor([hidden_dim])).to(device)

    def forward(self, trg, enc_src, trg_mask, src_mask):

        batch_size = trg.shape[0]
        trg_len = trg.shape[1]

        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)

        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))

        for layer in self.layers: 
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)

        output = self.fc_out(trg)

        return output, attention


## 5. 최종 Transformer 아키텍쳐
- 전체 Transformer 모델을 정의함.
- 입력을 들어왔을 때 앞서 정의한 인코더와 디코더를 거쳐 출력 문장을 생성함.

In [17]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_pad_idx, trg_pad_idx, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder

        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx

        self.device = device

    def make_src_mask(self, src):
        # unsqueeze는 텐서의 차원을 늘려주는 함수.
        # 예를 들어, (batch_size, src_len) 형태의 src에 대해 unsqueeze(1)을 하면 (batch_size, 1, src_len)이 되고,
        # 다시 unsqueeze(2)를 하면 (batch_size, 1, 1, src_len)이 돼.
        # 이렇게 차원을 늘려주는 이유는 이후에 어텐션 연산에서 브로드캐스팅이 잘 되도록 맞춰주기 위해서임.
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)
        
        return src_mask
    
    def make_trg_mask(self, trg):

        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        trg_len = trg.shape[1]

        trg_sub_mask = torch.tril(torch.ones(trg_len, trg_len, device = self.device)).bool()

        trg_mask = trg_pad_mask & trg_sub_mask

        return trg_mask
    
    def forward(self, src, trg):
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)

        enc_src = self.encoder(src, src_mask)

        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)

        return output, attention

In [18]:
# 모델 하이퍼파라미터 설정
INPUT_DIM = len(SRC)  # 새로운 vocab 구조에 맞게 수정
OUTPUT_DIM = len(TRG)  # 새로운 vocab 구조에 맞게 수정

print(f"입력 차원 (독일어 어휘 크기): {INPUT_DIM}")
print(f"출력 차원 (영어 어휘 크기): {OUTPUT_DIM}")
HIDDEN_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1
     

입력 차원 (독일어 어휘 크기): 7853
출력 차원 (영어 어휘 크기): 5893


In [19]:
# 패딩 인덱스 설정 (새로운 vocab 구조에 맞게)
SRC_PAD_IDX = SRC.stoi[SRC.pad_token]
TRG_PAD_IDX = TRG.stoi[TRG.pad_token]

print(f"Source 패딩 인덱스: {SRC_PAD_IDX}")
print(f"Target 패딩 인덱스: {TRG_PAD_IDX}")

# 인코더 & 디코더 & Transformer 객체 선언
enc = Encoder(INPUT_DIM, HIDDEN_DIM, ENC_LAYERS, ENC_HEADS, ENC_PF_DIM, ENC_DROPOUT, device)
dec = Decoder(OUTPUT_DIM, HIDDEN_DIM, DEC_LAYERS, DEC_HEADS, DEC_PF_DIM, DEC_DROPOUT, device)

model = Transformer(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)


Source 패딩 인덱스: 1
Target 패딩 인덱스: 1


- model 가중치 파라미터 초기화

In [20]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 9,038,341 trainable parameters


In [21]:
# 이 함수는 모델의 가중치(weight) 파라미터를 xavier_uniform 방식으로 초기화해주는 함수야.
# 신경망의 각 레이어(모듈) m에 대해, 만약 'weight' 속성이 있고 그 차원이 1보다 크면(즉, 선형 계층 등),
# nn.init.xavier_uniform을 사용해서 가중치를 초기화해. 이렇게 하면 학습 초기에 적절한 분포로 가중치가 설정되어
# 학습이 더 잘 되도록 도와줌.

def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)  # xavier_uniform_로 변경 (최신 권장 방식)

# 모델의 모든 서브모듈에 대해 위의 초기화 함수를 적용해.
model.apply(initialize_weights)

Transformer(
  (encoder): Encoder(
    (tok_embedding): Embedding(7853, 256)
    (pos_embedding): Embedding(100, 256)
    (layers): ModuleList(
      (0-2): 3 x EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attn): MultiHeadAttentionLayer(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc_1): Linear(in_features=256, out_features=512, bias=True)
          (fc_2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )


## 6. 학습 및 평가 함수 정의
- 모델 학습과 optimization 정의

In [22]:
import torch.optim as optim

lr = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr = lr)

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [23]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0

    for i, batch in enumerate(iterator):
        # 새로운 DataLoader 형태: 튜플 언패킹
        src, trg = batch
        
        # GPU로 데이터 이동
        src = src.to(device)
        trg = trg.to(device)

        optimizer.zero_grad()

        # 디코더 입력에서 마지막 토큰 제거 (teacher forcing)
        output, _ = model(src, trg[:, :-1])
        output_dim = output.shape[-1]

        # 출력을 1차원으로 변환
        output = output.contiguous().view(-1, output_dim)

        # 타겟에서 첫 번째 토큰(<sos>) 제거하고 1차원으로 변환
        trg = trg[:,1:].contiguous().view(-1)

        loss = criterion(output, trg)
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


In [24]:
def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for i, batch in enumerate(iterator):
            # 새로운 DataLoader 형태: 튜플 언패킹
            src, trg = batch
            
            # GPU로 데이터 이동
            src = src.to(device)
            trg = trg.to(device)

            # 디코더 입력에서 마지막 토큰 제거
            output, _ = model(src, trg[:,:-1])

            output_dim = output.shape[-1]
            # 출력을 1차원으로 변환
            output = output.contiguous().view(-1, output_dim)

            # 타겟에서 첫 번째 토큰(<sos>) 제거하고 1차원으로 변환
            trg = trg[:,1:].contiguous().view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [25]:
import math
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [26]:
import time
import math
import random

N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # 시작 시간 기록

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # 종료 시간 기록
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer_german_to_english.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):.3f}')
    print(f'\tValidation Loss: {valid_loss:.3f} | Validation PPL: {math.exp(valid_loss):.3f}')

Epoch: 01 | Time: 0m 7s
	Train Loss: 4.213 | Train PPL: 67.575
	Validation Loss: 3.071 | Validation PPL: 21.555
Epoch: 02 | Time: 0m 7s
	Train Loss: 2.801 | Train PPL: 16.465
	Validation Loss: 2.367 | Validation PPL: 10.667
Epoch: 03 | Time: 0m 7s
	Train Loss: 2.232 | Train PPL: 9.319
	Validation Loss: 2.041 | Validation PPL: 7.695
Epoch: 04 | Time: 0m 7s
	Train Loss: 1.886 | Train PPL: 6.592
	Validation Loss: 1.869 | Validation PPL: 6.480
Epoch: 05 | Time: 0m 7s
	Train Loss: 1.638 | Train PPL: 5.143
	Validation Loss: 1.778 | Validation PPL: 5.915
Epoch: 06 | Time: 0m 7s
	Train Loss: 1.452 | Train PPL: 4.273
	Validation Loss: 1.715 | Validation PPL: 5.557
Epoch: 07 | Time: 0m 7s
	Train Loss: 1.300 | Train PPL: 3.669
	Validation Loss: 1.686 | Validation PPL: 5.397
Epoch: 08 | Time: 0m 7s
	Train Loss: 1.175 | Train PPL: 3.239
	Validation Loss: 1.698 | Validation PPL: 5.463
Epoch: 09 | Time: 0m 7s
	Train Loss: 1.065 | Train PPL: 2.902
	Validation Loss: 1.701 | Validation PPL: 5.482
Epoch: